In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers

import skfuzzy as fuzz

from sklearn.metrics import mean_squared_error, mean_absolute_error
import arch


Importing data from yfinance

In [78]:
main_data=pdr.get_data_yahoo('CIPLA.NS',start='2010-06-29',end='2023-08-04')
main_data=main_data.reset_index()
main_data

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,348.399994,348.399994,340.000000,340.600006,314.235657,491114
1,2010-06-30,338.000000,341.700012,334.350006,338.500000,312.298157,1223210
2,2010-07-01,338.000000,339.799988,334.549988,335.250000,309.299652,417953
3,2010-07-02,334.500000,338.799988,334.500000,336.799988,310.729736,348768
4,2010-07-05,338.000000,342.899994,336.049988,337.299988,311.191040,711437
...,...,...,...,...,...,...,...
3230,2023-07-28,1171.449951,1190.400024,1155.699951,1177.800049,1177.800049,4454009
3231,2023-07-31,1181.800049,1184.750000,1165.400024,1175.000000,1175.000000,2221571
3232,2023-08-01,1175.000000,1187.300049,1168.099976,1178.349976,1178.349976,1362536
3233,2023-08-02,1185.000000,1185.000000,1153.550049,1168.300049,1168.300049,1861901


In [79]:
stock=main_data["Close"]

Importing the metrics dataframe

In [80]:
metrics = pd.read_csv("metrics.csv")

## ARIMA-ANN model

In [81]:
date = main_data.Date
close=main_data.Close

In [82]:
return_values = close.pct_change().dropna() # shape 3297 -->3296
return_values.shape

(3234,)

In [83]:
arma_order = (1, 0, 1)  
arma_model = sm.tsa.arima.ARIMA(return_values, order=arma_order)
arma_results = arma_model.fit()

c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [84]:
residual = arma_results.resid

Xt_1 = return_values.shift(1)
Xt_2 = return_values.shift(2)

# Lagged Values of the Residuals
et_1 = residual.shift(1)
et_2 = residual.shift(2)

In [85]:
data = pd.DataFrame({"Date": date[1:],
                            "Xt_1": Xt_1,
                            "Xt_2": Xt_2,
                            "et_1": et_1,
                            "et_2": et_2,
                            "Yt": return_values
                                 })
data.dropna(inplace=True)

data

,Date,Xt_1,Xt_2,et_1,et_2,Yt
3,2010-07-02,-0.009601,-0.006166,-0.010340,-0.006677,0.004623
4,2010-07-05,0.004623,-0.009601,0.003766,-0.010340,0.001485
5,2010-07-06,0.001485,0.004623,0.001112,0.003766,0.004744
6,2010-07-07,0.004744,0.001485,0.004266,0.001112,-0.012541
7,2010-07-08,-0.012541,0.004744,-0.012907,0.004266,0.014343
...,...,...,...,...,...,...
3230,2023-07-28,0.096350,0.013036,0.096267,0.012696,0.005421
3231,2023-07-31,0.005421,0.096350,0.008180,0.096267,-0.002377
3232,2023-08-01,-0.002377,0.005421,-0.002703,0.008180,0.002851
3233,2023-08-02,0.002851,-0.002377,0.002242,-0.002703,-0.008529


In [86]:
X = data[["Xt_1","Xt_2","et_1","et_2"]]
y = data['Yt']

In [87]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [88]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')


In [89]:

train_size = int(0.8 * len(X))  

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)


Epoch 1/50


C:\Users\chakr\AppData\Local\Temp\ipykernel_11360\161937034.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train, y_test = y[:train_size], y[train_size:]
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


73/73 [==============================] - 2s 6ms/step - loss: 0.0021 - val_loss: 0.0010
Epoch 2/50
73/73 [==============================] - 0s 3ms/step - loss: 3.5108e-04 - val_loss: 9.6869e-04
Epoch 3/50
73/73 [==============================] - 0s 4ms/step - loss: 3.0729e-04 - val_loss: 7.2336e-04
Epoch 4/50
73/73 [==============================] - 0s 3ms/step - loss: 2.9328e-04 - val_loss: 8.5265e-04
Epoch 5/50
73/73 [==============================] - 0s 4ms/step - loss: 2.7624e-04 - val_loss: 6.9544e-04
Epoch 6/50
73/73 [==============================] - 0s 3ms/step - loss: 2.7821e-04 - val_loss: 8.5594e-04
Epoch 7/50
73/73 [==============================] - 0s 3ms/step - loss: 3.1440e-04 - val_loss: 7.8805e-04
Epoch 8/50
73/73 [==============================] - 0s 3ms/step - loss: 2.8823e-04 - val_loss: 6.4560e-04
Epoch 9/50
73/73 [==============================] - 0s 3ms/step - loss: 2.9592e-04 - val_loss: 7.2568e-04
Epoch 10/50
73/73 [==============================] - 0s 3ms/step 

In [90]:
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)

21/21 [==============================] - 0s 2ms/step - loss: 2.7280e-04
Test Loss: 0.00027279520872980356


In [91]:
new_data = np.array([[-0.014049,0.034767,-0.015659,0.033157]])
new_data = scaler.transform(new_data)  
prediction = model.predict(new_data)
print("Predicted Value:", prediction[0][0])

1/1 [==============================] - 0s 99ms/step
Predicted Value: 0.0124743795


c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [92]:
y_pred = model.predict(X_test)

21/21 [==============================] - 0s 2ms/step


In [93]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Root Mean Squared Error (RMSE): 0.01651651303069767
Mean Absolute Error (MAE): 0.012318199202481365


In [94]:
arima_ann_metrics = pd.DataFrame([{'models':'ARIMA-ANN', 'RootMeanSquaredError':rmse, 'MeanAbsoluteError':mae}])

In [95]:
metrics = pd.concat([metrics,arima_ann_metrics ], ignore_index= True)
metrics

,models,RootMeanSquaredError,MeanAbsoluteError
0,Linear Regression(Multiple),3.632006,2.701710
1,"ARMA(2,2)",0.043155,0.030573
2,Sarima,0.043220,0.030645
3,AR(1),0.043154,0.030574
4,"GARCH(1,1)",0.043162,0.001863
5,"Arma-Garch(1,1)(1,1)",0.043180,0.030600
6,ARIMA-ANN,0.016517,0.012318


### ARMA-GARCH-ANN MODEL

In [96]:
stock = stock.pct_change().dropna()
stock

1      -0.006166
2      -0.009601
3       0.004623
4       0.001485
5       0.004744
          ...   
3230    0.005421
3231   -0.002377
3232    0.002851
3233   -0.008529
3234   -0.002097
Name: Close, Length: 3234, dtype: float64

In [97]:
arma_order = (1, 0, 1)  
arma_model = sm.tsa.arima.ARIMA(stock, order=arma_order)
arma_results = arma_model.fit()

c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [98]:
garch_order = (1, 1)  # Replace with the appropriate GARCH order (p, q)
garch_model = arch.arch_model(arma_results.resid, vol='Garch', p=garch_order[0], q=garch_order[1])
garch_results = garch_model.fit()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 5421249627.134298
Iteration:      2,   Func. Count:     18,   Neg. LLF: 978804747.5146753
Iteration:      3,   Func. Count:     30,   Neg. LLF: 3371745930.579838
Iteration:      4,   Func. Count:     43,   Neg. LLF: 2230558690.0203137
Optimization terminated successfully    (Exit mode 0)
            Current function value: -8822.648923743134
            Iterations: 5
            Function evaluations: 53
            Gradient evaluations: 4


c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002661. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [99]:
residuals = garch_results.resid

In [100]:
# Creating the dataframe for using ARMA_GARCH_ANN Model

# Lagged Values of the actual return price 
Xt_1 = stock.shift(1)
Xt_2 = stock.shift(2)


# Lagged Values of the Residuals
et_1 = residuals.shift(1)
et_2 = residuals.shift(2)

# Creating the dataframe
arma_garch_ann_df = pd.DataFrame({
    'Xt_1': Xt_1,
    'Xt_2': Xt_2,
    'et_1': et_1,
    'et_2': et_2,
    'Yt': stock
})

# Droping the null values 
arma_garch_ann_df.dropna(inplace=True)

#Printing the data
print(arma_garch_ann_df)

          Xt_1      Xt_2      et_1      et_2        Yt
3    -0.009601 -0.006166 -0.010405 -0.006741  0.004623
4     0.004623 -0.009601  0.003702 -0.010405  0.001485
5     0.001485  0.004623  0.001047  0.003702  0.004744
6     0.004744  0.001485  0.004202  0.001047 -0.012541
7    -0.012541  0.004744 -0.012972  0.004202  0.014343
...        ...       ...       ...       ...       ...
3230  0.096350  0.013036  0.096202  0.012632  0.005421
3231  0.005421  0.096350  0.008116  0.096202 -0.002377
3232 -0.002377  0.005421 -0.002768  0.008116  0.002851
3233  0.002851 -0.002377  0.002178 -0.002768 -0.008529
3234 -0.008529  0.002851 -0.009025  0.002178 -0.002097

[3232 rows x 5 columns]


In [101]:
X = arma_garch_ann_df[['Xt_1', 'Xt_2', 'et_1', 'et_2']]
y = arma_garch_ann_df['Yt']


scaler = StandardScaler()
X = scaler.fit_transform(X)


model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')


train_size = int(0.8 * len(X))  
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)


test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)


new_data = np.array([[-0.014049,0.034767,-0.015659,0.033157]])
new_data = scaler.transform(new_data)  
prediction = model.predict(new_data)
print("Predicted Value:", prediction[0][0])

y_pred = model.predict(X_test)

C:\Users\chakr\AppData\Local\Temp\ipykernel_11360\1117127597.py:19: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train, y_test = y[:train_size], y[train_size:]
c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/50
73/73 [==============================] - 2s 5ms/step - loss: 0.0111 - val_loss: 0.0015
Epoch 2/50
73/73 [==============================] - 0s 3ms/step - loss: 4.3973e-04 - val_loss: 9.8091e-04
Epoch 3/50
73/73 [==============================] - 0s 2ms/step - loss: 3.4550e-04 - val_loss: 8.4444e-04
Epoch 4/50
73/73 [==============================] - 0s 2ms/step - loss: 3.1660e-04 - val_loss: 8.3710e-04
Epoch 5/50
73/73 [==============================] - 0s 3ms/step - loss: 2.9825e-04 - val_loss: 7.4165e-04
Epoch 6/50
73/73 [==============================] - 0s 3ms/step - loss: 2.7386e-04 - val_loss: 7.9901e-04
Epoch 7/50
73/73 [==============================] - 0s 3ms/step - loss: 2.6346e-04 - val_loss: 7.3278e-04
Epoch 8/50
73/73 [==============================] - 0s 3ms/step - loss: 2.5913e-04 - val_loss: 6.7560e-04
Epoch 9/50
73/73 [==============================] - 0s 3ms/step - loss: 2.5757e-04 - val_loss: 7.0595e-04
Epoch 10/50
73/73 [==============================] - 0

c:\Users\chakr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [102]:
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse1)

# Calculate MAE
mae1 = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae1)

Root Mean Squared Error (RMSE): 0.01487900938893959
Mean Absolute Error (MAE): 0.010744818910452232


In [103]:
arima_garch_ann_metrics = pd.DataFrame([{'models':'ARIMA-GARCH-ANN', 'RootMeanSquaredError':rmse1, 'MeanAbsoluteError':mae1}])

In [104]:
metrics = pd.concat([metrics,arima_garch_ann_metrics], ignore_index= True)
metrics

,models,RootMeanSquaredError,MeanAbsoluteError
0,Linear Regression(Multiple),3.632006,2.701710
1,"ARMA(2,2)",0.043155,0.030573
2,Sarima,0.043220,0.030645
3,AR(1),0.043154,0.030574
4,"GARCH(1,1)",0.043162,0.001863
5,"Arma-Garch(1,1)(1,1)",0.043180,0.030600
6,ARIMA-ANN,0.016517,0.012318
7,ARIMA-GARCH-ANN,0.014879,0.010745


### FUZZY CLUSTERING

In [105]:
stock1 = main_data["Close"]
stock1 = stock1.values.reshape(-1,1)

In [106]:
num_clusters = 3

# Fuzzy C-Means clustering
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(stock1.T, num_clusters, 2, error=0.005, maxiter=1000)

# Identify the cluster with the highest membership for each data point
cluster_membership = np.argmax(u, axis=0)

In [107]:
main_data['Cluster'] = cluster_membership

In [ ]:
for cluster_num in range(num_clusters):
    # Filter data points belonging to the current cluster
    cluster_data = main_data[main_data['Cluster'] == cluster_num]
    x_train = cluster_data['Close'].values.reshape(-1, 1)
    y_train = cluster_data['Close'].shift(-1).fillna(0).values.reshape(-1, 1)


    model = keras.Sequential([
        layers.Dense(16, activation='relu', input_shape=(1,)),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=50, batch_size=16, verbose=0)
    
    # Example: Predict next day's stock price for the last data point in the cluster
    last_price = cluster_data['Close'].iloc[-1]
    predicted_price = model.predict(np.array([[last_price]]))
    print(f"Cluster {cluster_num}: Predicted Next Day's Price: {predicted_price[0][0]}")



plt.figure(figsize=(8, 6))
for i in range(num_clusters):
    plt.scatter(main_data[main_data['Cluster'] == i]['Date'], main_data[main_data['Cluster'] == i]['Close'], label=f'Cluster {i}')
plt.xlabel('Date')
plt.ylabel('Normalized Stock Price')
plt.title('Fuzzy C-Means Clustering of Tesla Stock Prices')
plt.legend()
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()
    